**1.1** Import the csv file into a dataframe. Analyze and understand each column properly.

In [84]:
import pandas as pd   # import the library
stock = pd.read_csv("data/GOOG.csv")  # import the CSV file

stock[stock.columns[1:-1]]=stock[stock.columns[1:-1]].replace('[,]','',regex=True)  #remove , from the string
stock[stock.columns[1:-1]]=stock[stock.columns[1:-1]].astype(float)   # convert string to float
print(stock.info())   # technical information about dataframe
print('---------------------------------------------------------')
print(stock.describe())   # quick overview of numerical datatypes
print('-------------------------------------------------------------------')
print(stock.head(7))  # first 7 rows
print('---------------------------------------------------------------------')
print(stock.tail(7))  # last 7 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   503 non-null    object 
 1   Close Price            503 non-null    float64
 2   Open Price             503 non-null    float64
 3   High Price             503 non-null    float64
 4   Low Price              503 non-null    float64
 5   Total Traded Quantity  503 non-null    object 
dtypes: float64(4), object(2)
memory usage: 23.7+ KB
None
---------------------------------------------------------
       Close Price   Open Price   High Price    Low Price
count   503.000000   503.000000   503.000000   503.000000
mean   1132.038767  1131.834433  1142.713996  1120.766163
std      70.086966    69.720548    68.474476    71.023314
min     984.670000   984.320000   994.430000   977.660000
25%    1073.645000  1073.960000  1084.985000  1062.375000
50%    1128.6300

**1.2** Calculate the maximum, minimum and mean price for the last 90 days. (assuming it's 90 trading days)

In [85]:
print(stock['Close Price'].head(90).max())  # head(90) gives us the data of last 90 trading days, and then max
print(stock['Close Price'].head(90).min())  # min of last 90 trading days
print(stock['Close Price'].head(90).mean()) # mean of last 90 trading days

1259.11
1076.63
1184.4741111111114


**1.3** Analyze data types of each column of dataframe. Convert object to **datetime64(ns)** format. And subtrac min from max of the **Date** column.

In [86]:
stock['Date']=stock['Date'].astype('datetime64')   # convert string to datetime format
print(stock.dtypes)   # print data types of the columns
print('---------------------------------------')
print(stock['Date'].max()-stock['Date'].min())  # max - min

Date                     datetime64[ns]
Close Price                     float64
Open Price                      float64
High Price                      float64
Low Price                       float64
Total Traded Quantity            object
dtype: object
---------------------------------------
729 days 00:00:00


**1.4** Calculate the monthwise VWAP (Volume Weighted Average Price) of the stock.<br>
**VWAP = sum(price*volume)/sum(volume)**<br>
We've created new columns **Month** and **Year** whose values are derived from the **Date** column. We've replaced **M** and **K** with  $10^{6}$ and $10^{3}$ respectively in the **Total Traded Quantity** column.

In [87]:
stock['Month'] = stock['Date'].dt.month
stock['Year'] = stock['Date'].dt.year
stock['Total Traded Quantity'] = stock['Total Traded Quantity'].replace({'K':'*1e3','M':'*1e6'},regex=True).map(pd.eval) # replace M with 1e6 and K with 1e3
stock['priceXvol'] = stock['Close Price']*stock['Total Traded Quantity']    # creating new column priceXvol for obtaining the sum(price*volume)
priceXvolume = stock.groupby(['Year','Month'])['priceXvol'].sum()   # sum(price*volume)
volume = stock.groupby(['Year','Month'])['Total Traded Quantity'].sum()   # sum(volume)
vwap = priceXvolume/volume
print(vwap)
# kindly suggest an alternate solution if this is not the correct way of doing what was asked.

Year  Month
2017  10       1020.769304
      11       1046.921972
      12       1052.501461
2018  1        1142.908136
      2        1088.801496
      3        1084.772690
      4        1038.839267
      5        1075.671110
      6        1150.524716
      7        1216.497232
      8        1241.468195
      9        1185.046656
      10       1110.061882
      11       1068.231777
      12       1044.025517
2019  1        1080.990612
      2        1125.534145
      3        1185.263631
      4        1232.921930
      5        1158.074381
      6        1077.036357
      7        1167.985066
      8        1180.733742
      9        1221.341273
      10       1222.747092
dtype: float64


**1.5** Write a function to calculate the average price over last N days.<br>
Write a function to calculate the profit/loss percentage over last N days.<br>
Calculate it over the course of last :
* 1 week
* 2 weeks
* 1 month
* 3 months
* 6 months
* 1 year






In [88]:
def averagePrice(N):
  return stock['Close Price'].head(N).mean()    # average price of last N trading days

def profitLossPercent(N):
  return (stock.iloc[0,1]-stock.iloc[N-1,1])%100    # profit loss percntage over last N trading days

def calculate(N,option):
  # error handling has not been taken care for the situation if the day was not a trading day, which could arise for option = 'day' or for date before that in csv file
  today = stock.iloc[0,0]
  m = today.month
  y = today.year
  d = today.day

  if(option=='day' or option=='days'):
    index = stock.index[stock['Date'] == today - pd.to_timedelta(N,unit='d')][0]

  if(option=='week' or option=='weeks'):
    index = stock.index[stock['Date'] == today - pd.to_timedelta(N,unit='w')][0]
  
  if(option=='month' or option=='months'):
    mminus = N % 12
    yminus = N // 12
    index = stock.index[(stock['Month'] == m - mminus) & (stock['Date'].dt.day == d+1 ) & (stock['Year'] == y - yminus)][0]
    
  if(option=='year' or option=='years'):   #assuming that N is not greater than 2019
    index = stock.index[(stock['Year'] == stock.iloc[0,0].year - N) & (stock['Date'].dt.day == d+1 ) & (stock['Month'] == m)][0]

  average = stock.iloc[0:index,1].mean()
  plpercent = ((stock.iloc[0,1] - stock.iloc[index,1])/stock.iloc[index,1])*100
  print("For ",N,option,",        Average :",average,",         Profit/Loss percent : ",plpercent,)

calculate(1,'week')
calculate(2,'weeks')
calculate(1,'month')
calculate(3,'months')
calculate(6,'months')
calculate(1,'year')
  

For  1 week ,        Average : 1249.3259999999998 ,         Profit/Loss percent :  0.503671775223495
For  2 weeks ,        Average : 1241.815 ,         Profit/Loss percent :  4.104277080043314
For  1 month ,        Average : 1223.9090476190477 ,         Profit/Loss percent :  1.057033243976423
For  3 months ,        Average : 1209.11671875 ,         Profit/Loss percent :  10.843002271246707
For  6 months ,        Average : 1170.6182677165357 ,         Profit/Loss percent :  -0.6493916391812787
For  1 year ,        Average : 1144.7866 ,         Profit/Loss percent :  14.092190034342464


**1.6** Add a column **Day_Perc_Change** where the values are daily change in percentages, the percentage change between two consecutive day's closing prices.

In [0]:
#since percentage change is calculated from previous day and the values in dataframe were in a reverse order, so adjusted it likewise
#stock['Day_Perc_Change'] = stock['Close Price'].pct_change()

stock['Day_Perc_Change'] = stock['Close Price'].pct_change(-1)
stock.loc[502,'Day_Perc_Change'] = 0

#print(stock.head())
#print(stock.tail())

**1.7** Add another column **Trend** based on **Day_Perc_Change**

In [0]:
stock.loc[(stock['Day_Perc_Change'] >= -0.5)&(stock['Day_Perc_Change'] < 0.5),'Trend'] = 'Slight or No change'
stock.loc[(stock['Day_Perc_Change'] >= 0.5)&(stock['Day_Perc_Change'] < 1),'Trend'] = 'Slight positive'
stock.loc[(stock['Day_Perc_Change'] >= -1)&(stock['Day_Perc_Change'] < -0.5),'Trend'] = 'Slight negative'
stock.loc[(stock['Day_Perc_Change'] >= 1)&(stock['Day_Perc_Change'] < 3),'Trend'] = 'Positive'
stock.loc[(stock['Day_Perc_Change'] >= -3)&(stock['Day_Perc_Change'] < -1),'Trend'] = 'Negative'
stock.loc[(stock['Day_Perc_Change'] >= 3)&(stock['Day_Perc_Change'] < 7),'Trend'] = 'Among top gainers'
stock.loc[(stock['Day_Perc_Change'] >= -7)&(stock['Day_Perc_Change'] < -3),'Trend'] = 'Among top losers'
stock.loc[(stock['Day_Perc_Change'] >= 7),'Trend'] = 'Bull run'
stock.loc[(stock['Day_Perc_Change'] < -7),'Trend'] = 'Bear drop'

**1.8** Find average and median values of column **Total Traded Quantity** for each types of **Trend**

In [99]:
print('Average',stock.groupby('Trend')['Total Traded Quantity'].mean(),sep='\n')
print('-----------------------------------------')
print('Median',stock.groupby('Trend')['Total Traded Quantity'].median(),sep='\n')

Average
Trend
Slight or No change    1.795025e+06
Name: Total Traded Quantity, dtype: float64
-----------------------------------------
Median
Trend
Slight or No change    1580000.0
Name: Total Traded Quantity, dtype: float64


**1.9** Save the modified dataframe to csv file.

In [0]:
stock.to_csv("output/week2.csv",index=False)